# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Training the bigram model

In [ ]:
# Your code here
import random
from tqdm import tqdm
import linecache
import random
from collections import defaultdict
from math import log10

In [22]:
prob_dict = {} # probability of words in different contexts
min_prob = 1.0
dictionary = defaultdict(lambda: 1) # frequency of words
all_words_counter = 0 # total number of words

In [23]:
def get_pair_from_bigrams_set():
  with open("./data/bigrams.txt", "r", encoding='latin-1') as file:
    for line in file:
      yield line.lower().split()

In [24]:
# TRAINING BIGRAMS
counter_without_words = {}
counter_with_words = {}
for value, context, word in get_pair_from_bigrams_set():
  value = int(value)
  if(context in counter_without_words):
      counter_without_words[context]+= value
  else:
      counter_without_words[context]= value

  if((word, context) in counter_with_words):
      counter_with_words[(word,context)]+= value
  else:
      counter_with_words[(word,context)]= value

  dictionary[word]+=value
  all_words_counter+=value

In [25]:
def get_pair_from_coca_set():
  with open("./data/coca_all_links.txt", "r", encoding='latin-1') as file:
    for line in file:
      yield line.lower().split()[:3]

In [26]:
for value, context, word in get_pair_from_coca_set():
  value = int(value)
  if(context in counter_without_words):
      counter_without_words[context]+= value
  else:
      counter_without_words[context]= value

  if((word, context) in counter_with_words):
      counter_with_words[(word,context)]+= value
  else:
      counter_with_words[(word,context)]= value

  dictionary[word]+=value
  all_words_counter+=value

In [27]:
# calculating probabilities of words in context
for key in counter_with_words.keys():
  if key[1] not in prob_dict:
     prob_dict[key[1]] = dict()

  prob_dict[key[1]][key[0]] = counter_with_words[key]/counter_without_words[key[1]]

  min_prob = min(min_prob, prob_dict[key[1]][key[0]])
min_prob = log10(min_prob) # log not to work with *, but with +-

### By this moment, the model is trained and ready to be used. Next are the function to use the model for correction

In [32]:
def get_prob(context, word):
  if context not in prob_dict and word in dictionary:
     return log10(dictionary[word] / all_words_counter)
  
  if context not in prob_dict or word not in prob_dict[context]:
     return log10(0.0001)+min_prob # sum of logs is product of their arguments, that are probabilities. so this value is always < min_prob

  return log10(prob_dict[context][word]) # log for better numbers

In [33]:
def edit_distance(s1,s2, drop_when = 2):
  # function to calculate edit distance
    if abs(len(s1) - len(s2)) > drop_when: # optimization, since we check edit_distance for word and every word in dict
      return drop_when+1

    dp = [[0 for j in range(len(s2)+1)] for i in range(len(s1)+1)]
    for i in range(1, len(s2)+1):
        dp[0][i] = i

    for i in range(1, len(s1)+1):
        dp[i][0] = i

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(min(dp[i-1][j],dp[i][j-1])+1, dp[i-1][j-1] + (1 if s1[i-1] != s2[j-1] else 0))

    return dp[len(s1)][len(s2)]

In [34]:
def get_corrected_word(context, word, next_word):
    # main function
    if context not in prob_dict:
        return word

    corrected_word = word

    max_prob = get_prob(context,word)+get_prob(word, next_word) # sum since we deal with logs, without log it is the product
    for cand in dictionary: #iterating over all words in dictionary
        if (edit_distance(cand, word) <= 2): # check only candidates with edit distance 2, otherwise too far
            if get_prob(context, cand) + get_prob(cand, next_word) > max_prob: # left is probability of word in context of 2 word. product in log gives sum
                # choose candidate with bigger probability
                max_prob = get_prob(context, cand) + get_prob(cand, next_word)
                corrected_word = cand

    return corrected_word

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### Justifications on training and algorithms itself
- I decided to use 2-grams, so only 1 word left and right is taken into account. Using larger n is not a big deal, but small n takes much less amount of data to train + less time + less space.
- The datasets are those that were given with the task: bigrams.txt and coca_all_links.txt.
- When working with probabilities, I used logarythms to avoid products of small float numbers. The technique is taken from one of the sources I was inspired by when writing.
- In general, training is the computation of probability of every word in dictionary to appear in different contexts.
- When model is asked to correct the word, it finds the new word (by iterating the whole dictionary) that is close (in terms of [edit distance](https://www.geeksforgeeks.org/edit-distance-dp-5/) that should be <=2) and has the highest probability of being the correct word given context (word before and after)
- Iterating over the whole dictionary to find close words is much better than iterating over all corrections of the word, since there are ~65k words in dictionary, while the word of even 6 symbols has >100k different misspelings.
- In case when context is not given or unknown, model gives higher probability to word that appears more frequently in the corpus.
- If the word to check probability is not known, model gives very small probability to such word to give a sense that this combination is nonsense.
### Nuances in testing
- The fivegrams.txt is used for testing: given 5 words, I introduce error in the second and fourth words and use them and their contexts as test cases.
- When generating test cases, there is 30% probability that the word will not be changed. It is done to check whether any algorithm will replace a correct word by incorrect (for example, "a" and "the" can be easily substituted by each other).
- As possible errors, I used the same code as in norvig solution: deletes, transposes, replaces and inserts.



## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

### Generating test set

In [14]:
# from norvig, used to generate error
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return [e2 for e1 in edits1(word) for e2 in edits1(e1)]

In [15]:
def generate_error(word, p_no_error = 0.3):
  # Here we generate spell error for given word.
  # with chance p_no_error we do not make any error
  # in other case, we generate error up to 2 edits
  if random.random() > 1 - p_no_error:
    return word
  possible_errors = edits2(word)
  return random.choice(possible_errors)

In [16]:
# here we generate test set
test_set_correct = []
test_set_errors = []
test_size = 50

for i in range(test_size):
  n_text, a, b, c, d, e = linecache.getline("./data/fivegrams.txt", random.randrange(0, 1040620)).split()
  test_set_correct.append([a,b,c,d,e])
  b,d = generate_error(b), generate_error(c)
  test_set_errors.append([a,b,c,d,e])


### Test of bigram model on test set

In [35]:
total = test_size * 2
correct = 0
for i in tqdm(range(len(test_set_errors))):
  a,b,c,d,e = test_set_errors[i]
  correction1 = get_corrected_word(a,b,c)
  if correction1 == test_set_correct[i][1]:
    correct +=1

  correction2 = get_corrected_word(c,d,e)
  if correction2 == test_set_correct[i][3]:
    correct +=1
print("\nAccuracy of bigrams is ", correct/total)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


Accuracy of bigrams is  0.45


### Test of norvig's solution on test set

In [20]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open("./data/big.txt").read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [31]:
total = test_size * 2
correct = 0
for i in tqdm(range(len(test_set_errors))):
  a,b,c,d,e = test_set_errors[i]
  correction1 = correction(b)
  if correction1 == test_set_correct[i][1]:
    correct +=1

  correction2 = correction(d)
  if correction2 == test_set_correct[i][3]:
    correct +=1
print("\nAccuracy of Norvig's solution is ", correct/total)

100%|██████████| 50/50 [00:01<00:00, 27.64it/s]


Accuracy of Norvig's solution is  0.32


## Comparison results
Norvig's solution give 32% of accuracy.\
Bigrams give 45% of accuracy, that is the improvement.